# VMware EXSi Visualisations 2

In [1]:
import pandas as pd
import json
import matplotlib.pyplot as plt
from matplotlib.colors import rgb2hex
from itertools import cycle
from IPython.core.display import HTML
from modules.aux_fun import *
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

In [2]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [3]:
json_file = 'data/machines.json' 
df = pd.read_json(json_file)
df

,Name,PowerState,Notes,Guest,NumCpu,CoresPerSocket,MemoryMB,MemoryGB,VMHostId,VMHost,...,GuestId,UsedSpaceGB,ProvisionedSpaceGB,DatastoreIdList,CreateDate,SEVEnabled,ExtensionData,CustomFields,Id,Uid
0,Controller,PoweredOn,,Controller:Ubuntu Linux (64-bit),1,1,1024,1,HostSystem-ha-host,esxi01.glasswall-icap.com,...,ubuntu64Guest,65.080298,129.079322,{Datastore-5fbce95c-733c4cab-0ac9-6c2b59c604ba},24/11/2020 11:46:16,,VMware.Vim.VirtualMachine,{},VirtualMachine-1,/VIServer=root@esxi01.glasswall-icap.com:443/V...
1,k8s-Sow,PoweredOff,Clone this VM if you would like to have a blan...,NaN,8,1,6144,6,HostSystem-ha-host,esxi01.glasswall-icap.com,...,ubuntu64Guest,2.474617,38.251752,{Datastore-5fbce95c-733c4cab-0ac9-6c2b59c604ba},25/11/2020 09:43:28,,VMware.Vim.VirtualMachine,{},VirtualMachine-10,/VIServer=root@esxi01.glasswall-icap.com:443/V...
2,ICAP_01,PoweredOff,Clone this VM if you would like to have a blan...,ICAP_01:,8,1,6144,6,HostSystem-ha-host,esxi01.glasswall-icap.com,...,ubuntu64Guest,17.732291,38.252590,{Datastore-5fbce95c-733c4cab-0ac9-6c2b59c604ba},08/12/2020 09:50:47,,VMware.Vim.VirtualMachine,{},VirtualMachine-303,/VIServer=root@esxi01.glasswall-icap.com:443/V...
3,haproxy-icap-wiki,PoweredOff,,haproxy-icap-wiki:,1,1,1024,1,HostSystem-ha-host,esxi01.glasswall-icap.com,...,ubuntu64Guest,12.300789,17.238000,{Datastore-5fbce95c-733c4cab-0ac9-6c2b59c604ba},11/12/2020 11:35:48,,VMware.Vim.VirtualMachine,{},VirtualMachine-310,/VIServer=root@esxi01.glasswall-icap.com:443/V...
4,haproxy-web-wiki,PoweredOff,,haproxy-web-wiki:,1,1,1024,1,HostSystem-ha-host,esxi01.glasswall-icap.com,...,ubuntu64Guest,12.381844,17.238000,{Datastore-5fbce95c-733c4cab-0ac9-6c2b59c604ba},11/12/2020 11:36:26,,VMware.Vim.VirtualMachine,{},VirtualMachine-311,/VIServer=root@esxi01.glasswall-icap.com:443/V...
5,import-proxy,PoweredOff,,import-proxy:,2,1,8192,8,HostSystem-ha-host,esxi01.glasswall-icap.com,...,ubuntu64Guest,14.320321,24.239964,{Datastore-5fbce95c-733c4cab-0ac9-6c2b59c604ba},11/12/2020 14:26:10,,VMware.Vim.VirtualMachine,{},VirtualMachine-316,/VIServer=root@esxi01.glasswall-icap.com:443/V...
6,minio-server,PoweredOff,,minio-server:,2,1,2048,2,HostSystem-ha-host,esxi01.glasswall-icap.com,...,ubuntu64Guest,48.828136,51.019810,{Datastore-5fbce95c-733c4cab-0ac9-6c2b59c604ba},01/01/1970 00:00:00,,VMware.Vim.VirtualMachine,{},VirtualMachine-382,/VIServer=root@esxi01.glasswall-icap.com:443/V...
7,file-gateway,PoweredOff,AWS-Appliance,file-gateway:,2,1,4096,4,HostSystem-ha-host,esxi01.glasswall-icap.com,...,otherGuest64,100.000008,104.191640,{Datastore-5fbce95c-733c4cab-0ac9-6c2b59c604ba},16/12/2020 14:00:39,,VMware.Vim.VirtualMachine,{},VirtualMachine-386,/VIServer=root@esxi01.glasswall-icap.com:443/V...
8,proxy-rebuild,PoweredOff,created by: Pranay,proxy-rebuild:,2,1,8192,8,HostSystem-ha-host,esxi01.glasswall-icap.com,...,ubuntu64Guest,12.691414,24.239964,{Datastore-5fbce95c-733c4cab-0ac9-6c2b59c604ba},16/12/2020 19:10:30,,VMware.Vim.VirtualMachine,{},VirtualMachine-397,/VIServer=root@esxi01.glasswall-icap.com:443/V...
9,GOV-UK,PoweredOff,,GOV-UK:,2,1,2048,2,HostSystem-ha-host,esxi01.glasswall-icap.com,...,ubuntu64Guest,20.288318,32.240188,{Datastore-5fbce95c-733c4cab-0ac9-6c2b59c604ba},17/12/2020 08:21:16,,VMware.Vim.VirtualMachine,{},VirtualMachine-409,/VIServer=root@esxi01.glasswall-icap.com:443/V...


## Option 1:  A subnetwork for each VM

In [4]:
# CREATE JSON FILE WITH NODES AND EDGES

network = Network()

network.groups = ['Name', 'GuestId', 'NumCpu', 'MemoryMB', 'PowerState', 
                   'Version', 'HardwareVersion', 'VMResourceConfiguration']


#colors = []
#for k in range(len(df)):
#    colors.append(rgb2hex(plt.cm.plasma(13*k)))

pastels = []
for k in range(9):
    pastels.append(rgb2hex(plt.cm.Pastel1(k)))
for k in range(8):
    pastels.append(rgb2hex(plt.cm.Pastel2(k)))

colors = []
for i, color in zip(range(len(df)), cycle(pastels)):
    colors.append(color)
    
    
for i, color in zip(range(len(df)), colors):
    # NODES   
    labels = []
    
    label = df.iloc[i]['Name'] + "\n" + df.iloc[i]['Id']
    labels.append(label)
    network.add_node(label, color=color, shape='box', repeat_nodes=True)
    
    for group in network.groups[1:-1]:
        label = group + ':\n' + str(df.iloc[i][group])
        labels.append(label)
        network.add_node(label, color=color, repeat_nodes=True)
        
    label = 'VMResourceConfiguration:\n' + wrap_by_word(df.iloc[i]['VMResourceConfiguration'], n=1)
    labels.append(label)
    network.add_node(label, color=color, shape='box', repeat_nodes=True)   
    
    # EDGES
    for label in labels[1:]:
        network.add_edge(network.nodesIds[labels[0]][-1], network.nodesIds[label][-1])
        

network.save_to_json('data/nodes_edges_3.json')

In [5]:
%%html
<div id="mynetwork"></div>

In [6]:
%%javascript
requirejs.config({
    paths: {
        vis: 'vis'
    }
});


require(['vis'], function(vis){ 
    
    var json = $.getJSON("data/nodes_edges_3.json")
      .done(function(data){
        var data = {
          nodes: data.nodes,
          edges: data.edges
        };
        var network = new vis.Network(container, data, options);
      });
       
    var options = {      
      width: '1200px',
      height: '1000px',
      locale: 'en',
      physics: true,
      interaction: {
        hover:true, 
        tooltipDelay: 300
      },        
      layout: {
        randomSeed: 1,
        improvedLayout: true,
      }
    };    
    
    var container = document.getElementById("mynetwork");
});

<IPython.core.display.Javascript object>

## Option 2: Connect all subnetworks to VMHost node

In [7]:
# CREATE JSON FILE WITH NODES AND EDGES

network = Network()

network.groups = ['Name', 'GuestId', 'NumCpu', 'MemoryMB', 'PowerState', 
                   'Version', 'HardwareVersion', 'VMResourceConfiguration']

    
name_labels = []
for i, color in zip(range(len(df)), colors):
    # NODES   
    labels = []
    
    label = df.iloc[i]['Name'] + "\n" + df.iloc[i]['Id']
    labels.append(label)
    name_labels.append(label)
    network.add_node(label, color=color, shape='box', repeat_nodes=True)
    
    for group in network.groups[1:-1]:
        label = group + ':\n' + str(df.iloc[i][group])
        labels.append(label)
        network.add_node(label, color=color, repeat_nodes=True)
        
    label = 'VMResourceConfiguration:\n' + wrap_by_word(df.iloc[i]['VMResourceConfiguration'], n=1)
    labels.append(label)
    network.add_node(label, color=color, shape='box', repeat_nodes=True)   
    
    # EDGES
    for label in labels[1:]:
        network.add_edge(network.nodesIds[labels[0]][-1], network.nodesIds[label][-1])
        
if is_unique(df['VMHost']):
    
    # Host Node
    host_label = 'VMHost:\n' + df.iloc[0]['VMHost']
    network.add_node(label=host_label, color="#5CD6D6", shape='box')
    
    # Conect Host node to all VMs
    for label in name_labels:
        network.add_edge(network.nodesIds[host_label][0], network.nodesIds[label][0], color="#5CD6D6", arrows="")       

network.save_to_json('data/nodes_edges_4.json')

In [8]:
%%html
<div id="mynetwork2"></div>

In [9]:
%%javascript
requirejs.config({
    paths: {
        vis: 'vis'
    }
});


require(['vis'], function(vis){ 
    
    var json = $.getJSON("data/nodes_edges_4.json")
      .done(function(data){
        var data = {
          nodes: data.nodes,
          edges: data.edges
        };
        var network = new vis.Network(container, data, options);
      });
       
    var options = {      
      width: '1200px',
      height: '1000px',
      locale: 'en',
      physics: true,
      interaction: {
        hover:true, 
        tooltipDelay: 300
      },        
      layout: {
        randomSeed: 1,
        improvedLayout: true,
      }
    };    
    
    var container = document.getElementById("mynetwork2");
});

<IPython.core.display.Javascript object>